<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Suicide_Ideation/blob/master/S_VS_T_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece
!pip install bert-for-tf2

     |████████████████████████████████| 1.2MB 7.1MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=a0f2948ed1514442442898e2a6e7011ac31b2d85ababfaa14cf8f597eea2816b
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=609ea0187224a7ecff162907dee212d7a127570ccb3fe984cce032e7a2fa7369
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=d851816572c8bfa0712be146fafa95623ad1414a2ac89f8a434746d602ed9be9
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [2]:
import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import os, re
import tensorflow_hub as hub
import tensorflow as tf
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
os.environ['KAGGLE_USERNAME'] = "nikhileswarkomati"
os.environ['KAGGLE_KEY'] = "001b3a30170775e55950edb6ff0c9b17"
!kaggle datasets download -d nikhileswarkomati/suicide-watch

 92% 106M/115M [00:00<00:00, 162MB/s] 
100% 115M/115M [00:00<00:00, 169MB/s]


In [4]:
!unzip '/content/suicide-watch.zip'

Archive:  /content/suicide-watch.zip
  inflating: SuicideAndDepression_Detection.csv  


In [5]:
data = pd.read_csv('/content/SuicideAndDepression_Detection.csv', lineterminator = '\n')
data.sample(5)

,text,class
268716,I've been contemplating it for a while nowI do...,SuicideWatch
289817,How common is it to get creepy PM's? I never g...,teenagers
159871,Kids who ask math teachers “How am I gonna use...,teenagers
205312,What kind of cars do y’all drive? Idk man put ...,teenagers
181930,I'm gonna sleep my way to death.I'm so tired. ...,SuicideWatch


In [6]:
print(data.shape)
data['class'].value_counts()

(348111, 2)


SuicideWatch    116037
depression      116037
teenagers       116037
Name: class, dtype: int64

In [8]:
# X = data.iloc[:, 0].values
# y = data.iloc[:, 1].values

# le = LabelEncoder()

# y = le.fit_transform(y)

# train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.5, stratify = y)
# train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size = 0.2, stratify = train_y)
# print(train_X.shape, test_y.shape, val_X.shape)

#------------------------------------------------------------------------------------------
X = data.loc[(data['class'] != 'depression'), 'text'].values
y = data.loc[(data['class'] != 'depression'), 'class'].values

le = LabelEncoder()

y = le.fit_transform(y)

train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.5, stratify = y)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size = 0.2, stratify = train_y)
print(train_X.shape, test_y.shape, val_X.shape)

(23207,) (116037,) (92830,)


In [11]:
from keras.layers import LSTM
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import itertools


%matplotlib inline


from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding
from keras.preprocessing import text, sequence
from keras import utils

embedding_vecor_length = 32
vocab_size = 10000
max_length = 600
oov_tok = '<OOV>'
trunc_type = 'post'

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_X)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_X)
padded = sequence.pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

model = Sequential()
model.add(Embedding(vocab_size, embedding_vecor_length, input_length = max_length))
model.add(LSTM(100, dropout = 0.3, recurrent_dropout = 0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())



checkpoint = tf.keras.callbacks.ModelCheckpoint('model_lstm.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
callbacks_list = [checkpoint, earlystopping]
model.fit(padded, train_y, epochs=10, batch_size=256,verbose = 1,callbacks = callbacks_list,validation_split = 0.1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 32)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 373,301
Trainable params: 373,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
82/82 [==============================] - 248s 3s/step - loss: 0.5773 - accuracy: 0.6927 - val_loss: 0.3485 - val_accuracy: 0.8755

Epoch 00001: val_accuracy improved from -inf to 0.87548, saving model to model_lstm.h5
Epoch 2/10
82/82 [==============================] - 243s 3s/step - loss: 0.2990 - accuracy: 0.8912 - val_loss: 0.3080 - val_accuracy: 0.8832

Ep

In [13]:
from keras.models import load_model
model = load_model('/content/model_lstm.h5')

In [14]:
testing_sequences = tokenizer.texts_to_sequences(test_X)
testing_padded = sequence.pad_sequences(testing_sequences, maxlen = max_length)
y_pred = model.predict(testing_padded)
print(accuracy_score(np.rint(y_pred), test_y))
print(classification_report(np.rint(y_pred), test_y))

0.924782612442583
              precision    recall  f1-score   support

         0.0       0.93      0.92      0.92     58320
         1.0       0.92      0.93      0.92     57717

    accuracy                           0.92    116037
   macro avg       0.92      0.92      0.92    116037
weighted avg       0.92      0.92      0.92    116037

